In [25]:
import json
import uproot
import numpy as np
import pandas as pd
import mplhep as mh

from pathlib import Path
from typing import Optional, Union
from hist import Hist, intervals
from hist.axis import Regular
from datetime import datetime
from datetime import timedelta

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colors import LogNorm
from matplotlib.patches import Rectangle

In [26]:
def get_region_params(region: str) -> dict:
    facecolor_table = {
        'All': ['#8fbdc4', '#5ab2bf', '#00aec9'],
        'Barrel': ['#87c7a7', '#4ac285', '#02c260'],
        'Disk1,2,3': ['#9498d4', '#555dd4', '#020dcc'],
        'Disk4': ['#fab4a2', '#fa7655', '#FF3300']
    }
    edgecolor_table = {
        'All': ['#005F77', '#005F77', '#005F77'],
        'Barrel': ['#003b0f', '#003b0f', '#003b0f'],
        'Disk1,2,3': ['#000775', '#000775', '#000775'],
        'Disk4': ['#CC0000', '#CC0000', '#CC0000']
    }
    hatches = ['\\\\', '//', 'oo']
    is_region = np.vectorize(lambda item: item.startswith(region))
    
    if region == 'All':
        facecolors = facecolor_table[region]
        edgecolors = edgecolor_table[region]
        is_region = np.vectorize(lambda item: type(item) is str)
    elif region == 'Barrel':
        facecolors = facecolor_table[region]
        edgecolors = edgecolor_table[region]
        is_region = np.vectorize(lambda item: item.startswith('W'))
    elif region == 'Endcap':
        facecolors = facecolor_table['Disk1,2,3']
        edgecolors = edgecolor_table['Disk1,2,3']
        is_region = np.vectorize(lambda item: item.startswith('RE'))
    elif region == 'Disk1,2,3':
        facecolors = facecolor_table[region]
        edgecolors = edgecolor_table[region]
        is_region = np.vectorize(lambda item: item.startswith(('RE+1', 'RE+2', 'RE+3', 'RE-1', 'RE-2', 'RE-3')))
    elif region == 'Disk4':
        facecolors = facecolor_table[region]
        edgecolors = edgecolor_table[region]
        is_region = np.vectorize(lambda item: item.startswith(('RE+4', 'RE-4')))
    elif region.startswith('W'):
        facecolors = facecolor_table['Barrel']
        edgecolors = edgecolor_table['Barrel']
    elif region.startswith(('RE+1', 'RE+2', 'RE+3', 'RE-1', 'RE-2', 'RE-3')):
        facecolors = facecolor_table['Disk1,2,3']
        edgecolors = edgecolor_table['Disk1,2,3']
    elif region.startswith(('RE+4', 'RE-4')):
        facecolors = facecolor_table['Disk4']
        edgecolors = edgecolor_table['Disk4']
    else:
        facecolors = facecolor_table['All']
        edgecolors = edgecolor_table['All']

    return {
        'is_region': is_region,
        'facecolors': facecolors,
        'edgecolors': edgecolors,
        'hatches': hatches
    }


def hist_eff_by_roll(input_path1, input_path2, input_path3, region, output_path):
    region_params = get_region_params(region)
    is_iRPC = np.vectorize(lambda roll: roll in {"RE+4_R1_CH15_A", "RE+4_R1_CH16_A", "RE+3_R1_CH15_A", "RE+3_R1_CH16_A"})

    total_by_roll1 = uproot.open(f'{input_path1}:total_by_roll').to_hist()
    passed_by_roll1 = uproot.open(f'{input_path1}:passed_by_roll').to_hist()
    
    total1 = total_by_roll1.values()
    passed1 = passed_by_roll1.values()
    roll_name1 = np.array(total_by_roll1.axes[0])

    total1 = total1[region_params['is_region'](roll_name1) & ~is_iRPC(roll_name1)]
    passed1 = passed1[region_params['is_region'](roll_name1) & ~is_iRPC(roll_name1)]

    eff1 = np.divide(
        passed1,
        total1,
        out=np.zeros_like(total1),
        where=(total1 > 0)
    ) * 100

    eff1 = eff1[total1 != 0] 
    n_total1 = len(total1)
    n_under70_1 = len(eff1[eff1 <= 70])
    n_excluded1 = len(total1[total1 == 0])

    total_by_roll2 = uproot.open(f'{input_path2}:total_by_roll').to_hist()
    passed_by_roll2 = uproot.open(f'{input_path2}:passed_by_roll').to_hist()

    total2 = total_by_roll2.values()
    passed2 = passed_by_roll2.values()
    roll_name2 = np.array(total_by_roll2.axes[0])

    total2 = total2[region_params['is_region'](roll_name2) & ~is_iRPC(roll_name2)]
    passed2 = passed2[region_params['is_region'](roll_name2) & ~is_iRPC(roll_name2)]

    eff2 = np.divide(
        passed2,
        total2,
        out=np.zeros_like(total2),
        where=(total2 > 0)
    ) * 100

    eff2 = eff2[total2 != 0]
    n_total2 = len(total2)
    n_under70_2 = len(eff2[eff2 <= 70])
    n_excluded2 = len(total2[total2 == 0])

    total_by_roll3 = uproot.open(f'{input_path3}:total_by_roll').to_hist()
    passed_by_roll3 = uproot.open(f'{input_path3}:passed_by_roll').to_hist()
    
    total3 = total_by_roll3.values()
    passed3 = passed_by_roll3.values()
    roll_name3 = np.array(total_by_roll3.axes[0])

    total3 = total3[region_params['is_region'](roll_name3) & ~is_iRPC(roll_name3)]
    passed3 = passed3[region_params['is_region'](roll_name3) & ~is_iRPC(roll_name3)]

    eff3 = np.divide(
        passed3,
        total3,
        out=np.zeros_like(total3),
        where=(total3 > 0)
    ) * 100

    eff3 = eff3[total3 != 0] 
    n_total3 = len(total3)
    n_under70_3 = len(eff3[eff3 <= 70])
    n_excluded3 = len(total3[total3 == 0])

    mh.style.use(mh.styles.CMS)
    fig, ax = plt.subplots(figsize=(12, 8))
    mh.cms.label(ax=ax, data=True, llabel='Work in Progress', loc=0,
                 year='Run 3', com=13.6, fontsize=24)
    ax.set_xlabel('Efficiency [%]', fontsize=24)
    ax.set_ylabel('Number of Rolls', fontsize=24)
    ax.set_xlim(70, 100)
    ax.annotate(
        f'Tag-and-Probe method',
        (0.96, 0.90),
        #weight='bold',
        xycoords='axes fraction',
        fontsize=24,
        horizontalalignment='right'
    )
    
    ax.annotate(
        f'RPC {region}',
        (0.04, 0.90),
        #weight='bold',
        xycoords='axes fraction',
        fontsize=24,
        horizontalalignment='left'
    )

    count1, bins1, patch1 = ax.hist(
        eff1[eff1 > 0], 
        bins=200, 
        range=(0, 100),
        facecolor=region_params['facecolors'][0],
        edgecolor=region_params['edgecolors'][0],
        hatch=region_params['hatches'][0],
        alpha=1.0,
        align='mid',
        density=False,
        linewidth=2.0,
        histtype='stepfilled'
    )

    count2, bins2, patch2 = ax.hist(
        eff2[eff2 > 0], 
        bins=200,
        range=(0, 100),
        facecolor=region_params['facecolors'][1],
        edgecolor=region_params['edgecolors'][1],
        hatch=region_params['hatches'][1],
        alpha=1.0,
        align='mid',
        density=False,
        linewidth=2.0,
        histtype='stepfilled'
    )

    count3, bins3, patch3 = ax.hist(
        eff3[eff3 > 0], 
        bins=200,
        range=(0, 100),
        facecolor=region_params['facecolors'][2],
        edgecolor=region_params['edgecolors'][2],
        hatch=region_params['hatches'][2],
        alpha=1.0,
        align='mid',
        density=False,
        linewidth=2.0,
        histtype='stepfilled'
    )

    ax.set_ylim(0, np.max(np.concatenate((count1, count2, count3)) * 1.2))
    
    extra = Rectangle((0, 0), 0.1, 0.1, fc='w', fill=False, edgecolor='none', linewidth=0)
    legend_header = [
        '',
        '',
        r'$Mean\ (>70\ \%)$',
        r'$\%\ (\leq70\ \%)$',
    ]
    legend_row1 = [
        '', 
        r'$2022\ (34.7\ fb^{-1})$', 
        f'       {np.mean(eff1[eff1 > 70]):.1f} %',
        f'     {n_under70_1 / n_total1 * 100:.1f} %',
    ]
    legend_row2 = [
        '', 
        r'$2023\ (27.9\ fb^{-1})$', 
        f'       {np.mean(eff2[eff2 > 70]):.1f} %',
        f'     {n_under70_2 / n_total2 * 100:.1f} %',
    ]
    legend_row3 = [
        '', 
        r'$2024\ (52.4\ fb^{-1})$', 
        f'       {np.mean(eff3[eff3 > 70]):.1f} %',
        f'     {n_under70_3 / n_total3 * 100:.1f} %',
    ]
    
    legend_handles, legend_values = [], []
    for idx in range(len(legend_header)):
        if idx == 0:
            legend_handles += [extra, patch1[0], patch2[0], patch3[0]]
        else:
            legend_handles += [extra, extra, extra, extra]
        legend_values += [legend_header[idx], legend_row1[idx], legend_row2[idx], legend_row3[idx]]

    ax.legend(
        legend_handles, legend_values,
        ncol=len(legend_header), columnspacing=-0.6,
        handletextpad=-0.3, handlelength=2.0, handleheight=1.6,
        alignment='center', loc=(0.03, 0.55), fontsize=18
    )
    
    output_path = Path(output_path)
    if not output_path.parent.exists():
        output_path.parent.mkdir(parents=True)
    fig.savefig(output_path)
    plt.close(fig)
    return [eff1, eff2, eff3]

In [27]:
input_path2022 = Path('/users/hep/eigen1907/Workspace/Workspace-RPC/Log/NanoAOD-TnP/240923/merge/Run2022.root')
input_path2023 = Path('/users/hep/eigen1907/Workspace/Workspace-RPC/Log/NanoAOD-TnP/240923/merge/Run2023.root')
input_path2024 = Path('/users/hep/eigen1907/Workspace/Workspace-RPC/Log/NanoAOD-TnP/240923/merge/Run2024.root')

workspace = Path('/users/hep/eigen1907/Workspace/Workspace-RPC/Log/NanoAOD-TnP/240923/plot/')

In [28]:
regions = ['Barrel', 'Endcap']
for region in regions:
    hist_eff_by_roll(
        input_path2022,
        input_path2023,
        input_path2024,
        region,
        workspace / ('tnp_eff_hist_run3_' + region + '.png')
    )